In [1]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
v = embedding_model.encode(user_question)

In [59]:
print(v[0])

0.078222655


In [4]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [5]:
filtered_documents = [doc for doc in documents if doc.get('course') == 'machine-learning-zoomcamp']
len(filtered_documents)

375

In [6]:
import numpy as np
from tqdm.notebook import tqdm

In [7]:
embeddings = []
for doc in tqdm(filtered_documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embeddings.append(embedding_model.encode(qa_text))
X = np.array(embeddings)

  0%|          | 0/375 [00:00<?, ?it/s]

In [8]:
X.shape

(375, 768)

In [9]:
scores = X.dot(v)

In [10]:
max(scores)

0.6506573

In [11]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        #idx = np.argsort(-scores)[:num_results]
        idx = np.argpartition(-scores, num_results)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
results = search_engine.search(v, num_results=5)

In [12]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [13]:
print(ground_truth[0])

{'question': 'Where can I sign up for the course?', 'course': 'machine-learning-zoomcamp', 'document': '0227b872'}


In [14]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [15]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    q_encoded = embedding_model.encode(q['question'])
    results = search_engine.search(q_encoded, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

hr = hit_rate(relevance_total)
print(f'Hit rate is {hr}')

  0%|          | 0/1830 [00:00<?, ?it/s]

Hit rate is 0.9398907103825137


In [19]:
for doc, emb in zip(filtered_documents, X):
    doc['question_text_vector'] = emb

In [49]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_2407/1973043213.py:30: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [50]:
for doc in tqdm(filtered_documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [51]:
query = {
    "field": "question_text_vector",
    "query_vector": v,
    "k": 5,
    "num_candidates": 10000, 
}

res = es_client.search(index=index_name, knn=query, source=["text", "question", "id"])

In [56]:
print(res['hits']['hits'][0]['_source']['id'])

ee58a693


In [61]:
#calculating hit rate for elasticsearch
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    q_encoded = embedding_model.encode(q['question'])
    query = {
        "field": "question_text_vector",
        "query_vector": q_encoded,
        "k": 5,
        "num_candidates": 100, 
    }
    results = es_client.search(index=index_name, knn=query, source=["id"])['hits']['hits']
    relevance = [d['_source']['id'] == doc_id for d in results]
    relevance_total.append(relevance)

hr = hit_rate(relevance_total)
print(f'Hit rate is {hr}')

  0%|          | 0/1830 [00:00<?, ?it/s]

Hit rate is 0.9398907103825137
